# Project: Tidal transformation of M33 
# Evolution of the internal stellar structure of M33 

## Numerical analysis of disk thickness

In [1]:
# Marco Barragan
# ASTR400B
#
# This code is to make a class that can be called to find and plot the disk thickness 
# of a text file. The idea is to use this 

In [2]:
# import modules
import numpy as np
import astropy.units as u
from astropy.constants import G

# import plotting modules
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

# my modules
from ReadFile import Read
from CenterOfMass import CenterOfMass
from MassProfile import MassProfile

In [14]:
# Create a class that outputs that returns an average disk thickness
# Possibly being able to output in terms of phi

class DiskProfile():
    
    def __init__(self):
        # Put in initial conditions
        # Create a COM of object for M31 Disk Using Code from Assignment 4
        COMD = CenterOfMass("M33_000.txt",2)

        # Compute COM of M31 using disk particles
        COMP = COMD.COM_P(0.1)
        COMV = COMD.COM_V(COMP[0],COMP[1],COMP[2])

        # Initialize COM position and velocity
        # Determine positions of disk particles relative to COM 
        xD = COMD.x - COMP[0].value 
        yD = COMD.y - COMP[1].value 
        zD = COMD.z - COMP[2].value 

        # total magnitude
        self.rtot = np.sqrt(xD**2 + yD**2 + zD**2)

        # Determine velocities of disk particles relatiev to COM motion
        vxD = COMD.vx - COMV[0].value 
        vyD = COMD.vy - COMV[1].value 
        vzD = COMD.vz - COMV[2].value 

        # total velocity 
        self.vtot = np.sqrt(vxD**2 + vyD**2 + vzD**2)

        # Vectors for r and v 
        self.r = np.array([xD,yD,zD]).T # transposed 
        self.v = np.array([vxD,vyD,vzD]).T
    
    

    # a function that will rotate the position and velocity vectors
    # so that the disk angular momentum is aligned with z axis. 

    def RotateFrame(self,posI,velI):
        # input:  3D array of positions and velocities
        # returns: 3D array of rotated positions and velocities such that j is in z direction

        # compute the angular momentum
        L = np.sum(np.cross(posI,velI), axis=0)
        # normalize the vector
        L_norm = L/np.sqrt(np.sum(L**2))

        # Set up rotation matrix to map L_norm to z unit vector (disk in xy-plane)

        # z unit vector
        z_norm = np.array([0, 0, 1])

        # cross product between L and z
        vv = np.cross(L_norm, z_norm)
        s = np.sqrt(np.sum(vv**2))

        # dot product between L and z 
        c = np.dot(L_norm, z_norm)

        # rotation matrix
        I = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
        v_x = np.array([[0, -vv[2], vv[1]], [vv[2], 0, -vv[0]], [-vv[1], vv[0], 0]])
        R = I + v_x + np.dot(v_x, v_x)*(1 - c)/s**2

        # Rotate coordinate system
        pos = np.dot(R, posI.T).T
        vel = np.dot(R, velI.T).T

        return pos, vel
    
    
    # Make a function that uses the above to make the plane of M33 flat
    
    def M33Plane(self):
        '''
        
        '''
        return
        
    # Make a function that defines the sigma radus (or x,y,z) from COM
    # in order to be able to use it as a cuttoff. This cutoff is needed 
    # because it's hard to choose a stopping point to measure H (thickness)
    def SigmaBound(self):
        '''
        
        '''
        return
        
    # Make a function that uses cuttoff and calculates average disk thickness
    def AverageThickness(self):
        '''
        
        '''
        return
    
    # Make a function that outputs values 

In [7]:
def RotateFrame(posI,velI):
    # input:  3D array of positions and velocities
    # returns: 3D array of rotated positions and velocities such that j is in z direction

    # compute the angular momentum
    L = np.sum(np.cross(posI,velI), axis=0)
    # normalize the vector
    L_norm = L/np.sqrt(np.sum(L**2))

    # Set up rotation matrix to map L_norm to z unit vector (disk in xy-plane)

    # z unit vector
    z_norm = np.array([0, 0, 1])

    # cross product between L and z
    vv = np.cross(L_norm, z_norm)
    s = np.sqrt(np.sum(vv**2))

    # dot product between L and z 
    c = np.dot(L_norm, z_norm)

    # rotation matrix
    I = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    v_x = np.array([[0, -vv[2], vv[1]], [vv[2], 0, -vv[0]], [-vv[1], vv[0], 0]])
    R = I + v_x + np.dot(v_x, v_x)*(1 - c)/s**2

    # Rotate coordinate system
    pos = np.dot(R, posI.T).T
    vel = np.dot(R, velI.T).T

    return pos, vel

In [10]:
# Create a function according to approximate the thin and thick disk height (Sparke and Gallagher 2006, Reid, Knude)
def ThinThick(z,R,n0,zthin,zthick,rs):
    """
    Input:
        z is the height above the disk
        R is the radius of the particle
        n0 is the initial star density
        zthin is the approximation for the scale height of the thin disk
        zthick is the scale approximation for the thick disk
        rs is the scale radius of the disk
    Returns:
        Outputs a set list of values n to go with each corresponding particle
    """
    
    nthin = n0 * np.exp(-z/zthin) * np.exp(-R/rs)
    nthick = n0 * 0.085* np.exp(-z/zthick) * np.exp(-R/rs)
    
    return nthin, nthick

In [11]:
# Make plots to make a movie
for i in range(10):
    # Compose the data filename (be careful about the folder)
    # Add a string of the filenumber to "000"
    ilbl = '000' + str(i)
    # Remove all but the last 3 digits
    ilbl = ilbl[-3:]
    filename = "M33_" + ilbl + '.txt'
    
    # Make filename according to Collin's naming scheme for ffpmeg
    outfile = f'plot_{i:03}.png'
    
    # Create a COM of object for M31 Disk Using Code from Assignment 4
    COMD = CenterOfMass(filename,2)

    # Compute COM of M31 using disk particles
    COMP = COMD.COM_P(0.1)
    COMV = COMD.COM_V(COMP[0],COMP[1],COMP[2])

    # Initialize COM position and velocity
    # Determine positions of disk particles relative to COM 
    xD = COMD.x - COMP[0].value 
    yD = COMD.y - COMP[1].value 
    zD = COMD.z - COMP[2].value 

    # total magnitude
    rtot = np.sqrt(xD**2 + yD**2 + zD**2)

    # Determine velocities of disk particles relatiev to COM motion
    vxD = COMD.vx - COMV[0].value 
    vyD = COMD.vy - COMV[1].value 
    vzD = COMD.vz - COMV[2].value 

    # total velocity 
    vtot = np.sqrt(vxD**2 + vyD**2 + vzD**2)

    # Vectors for r and v 
    r = np.array([xD,yD,zD]).T # transposed 
    v = np.array([vxD,vyD,vzD]).T
    
    # Rotate vectors so that the disk angular momentum is aligned with z axis
    rn, vn = RotateFrame(r,v)
    
    # Store the radius value
    rntot = np.sqrt(r[:,0]**2 + r[:,1]**2 + r[:,2]**2)
    
    # Store the values of n
    nthin, nthick = ThinThick(r[:,2],rntot,1,0.3,1,8)
    
    # The goal here is to plot n vs z, and then with the points we see, try to approximate zthin and zthick accordingly.
    # Plot n vs z
    fig,ax = plt.subplots(figsize=(10,8))

    #adjust tick label font size
    label_size = 22
    matplotlib.rcParams['xtick.labelsize'] = label_size 
    matplotlib.rcParams['ytick.labelsize'] = label_size

    # Plot 
    plt.semilogy(r[:,2], nthin, '--', linewidth=3, label='Thin Disk')
    plt.semilogy(r[:,2], nthick, '--', linewidth=3, label='Thick Disk')
    plt.semilogy(r[:,2], nthin + nthick, '--', linewidth=3, label='Approximation')

    # Axes labels 
    plt.title('Orbit of Galaxies',fontsize=22)
    plt.xlabel('t (Gyr)',fontsize=22) 
    plt.ylabel('r (kpc)', fontsize=22)

    # Legend
    plt.legend(loc='upper right',fontsize='x-large')


    
    plt.savefig(outfile)
    plt.close()
    
    print(i)
    
    '''fig = plt.figure()
    DPI = fig.get_dpi()
    fig.set_size_inches(1200/float(DPI),610.0/float(DPI))'''

0
1
2
3
4
5
6
7
8
9
